In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
!pip install transformers==4.15.0 sentencepiece
!pip install datasets==1.17.0 
!pip install langid
!pip install jiwer
! pip install pythainlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 5.0 MB/s 


In [4]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from transformers import AutoModelForMaskedLM, pipeline
from transformers import AutoTokenizer, BertForTokenClassification
from tqdm import tqdm 
import pickle
from datasets import load_metric
import subprocess
import sys
import langid
from nltk.translate.gleu_score import sentence_gleu
from jiwer import cer
from pythainlp import correct

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
tokenizer = pickle.load(open('drive/MyDrive/AIBuilders/json/tokenizer_json_15k.pkl', 'rb'))

In [10]:
ds_mlm = pickle.load(open('drive/MyDrive/AIBuilders/json/mlm_json_15k.pkl', 'rb'))
ds_tag = pickle.load(open('drive/MyDrive/AIBuilders/json/ner_json_15k.pkl', 'rb'))

In [11]:
ds_mlm = ds_mlm[10576:]
ds_tag = ds_tag[10576:]
NUM_SAMPLE = ds_mlm.shape[0]
NUM_SAMPLE

5000

In [ ]:
bleu = load_metric("bleu")
glue = load_metric("glue", "mrpc")
totalacc = 0
bleu_higher = 0
bleu_lower = 0
bleu_equal = 0
total_f1 = 0
total_gs_ori = 0
total_gs_pred = 0
total_cer_ori = 0
total_cer_pred = 0

# NUM_SAMPLE=10
for i in tqdm(range(NUM_SAMPLE)):
  text_id = ds_tag.iloc[i]['text']['input_ids'].squeeze(0).tolist()
  text_id = [k for k in text_id if k != 1]
  text_token = tokenizer.convert_ids_to_tokens(text_id)
  original_token = text_token.copy()
  text_token_len = len(text_token)
  original = [original_token]
  labels_id = ds_mlm.iloc[i]['labels']
  labels_id = [k for k in labels_id if k != 1]
  labels_token = tokenizer.convert_ids_to_tokens(labels_id)
  references = [[labels_token]]
  # print(text_id)
  # print(labels_id)

  # print(f"NO. {i}, {''.join(text_token)}")

  for j in range(text_token_len):
    text_token[j] = text_token[j].replace("▁", " ")
    try:
      corrected = correct(text_token[j])
    except:
      print(f"err pythainlp: {text_token[j]}")
    else:
      # print(f"{text_token[j]} => {corrected}")
      text_token[j] = corrected
    
  predictions = [text_token]
  final = "".join(text_token)
  final = final.replace("▁", " ")

  numer = 0
  denom = 0
  TP = 0
  FP = 0
  TN = 0
  FN = 0
  # print(len(text_token), len(labels_token))
  # print(text_token)
  # print(labels_token)
  labels_len = len(labels_token)
  for i in range(labels_len):
    if not text_token[i] == original_token[i]: #change
      denom += 1
      if text_token[i] == labels_token[i]:
        numer += 1
        TP += 1
      elif not text_token[i] == labels_token[i]:
        FP += 1
    elif text_token[i] == original_token[i]: #no change
      if text_token[i] == labels_token[i]:
        TN += 1
      elif not text_token[i] == labels_token[i]:
        FN += 1
  if denom == 0:
    acc = 0
  else:
    acc = float(numer)/float(denom)
  totalacc += acc

  # print(f"TP:{TP}   TN:{TN}   FP:{FP}   FN:{FN}")
  precision = float(TP) / float(TP+FP) if TP+FP > 0 else 0
  recall = float(TP) / float(TP+FN) if TP+FN > 0 else 0
  f1 = float(2*precision*recall) / float(precision + recall) if precision+recall > 0 else 0
  total_f1 += f1

  references[0][0] = [k.replace("▁", " ") for k in references[0][0]]
  original[0] = [k.replace("▁", " ") for k in original[0]]
  predictions[0] = [k.replace("▁", " ") for k in predictions[0]]

  bleu_original = bleu.compute(predictions=original, references=references)
  bleu_prediction = bleu.compute(predictions=predictions, references=references)

  if bleu_prediction['bleu'] > bleu_original['bleu']:
    bleu_higher += 1
  elif bleu_prediction['bleu'] < bleu_original['bleu']:
    bleu_lower += 1
  elif bleu_prediction['bleu'] == bleu_original['bleu']:
    bleu_equal += 1

  # print(references)
  # print(predictions)
  # print(original)
  gleu_score = sentence_gleu(references[0], predictions[0], min_len=1, max_len=4)
  total_gs_pred += gleu_score
  gleu_score_original = sentence_gleu(references[0], original[0], min_len=1, max_len=4)
  total_gs_ori += gleu_score_original

  cer_text_ori = "".join(original[0])
  cer_text_pred = "".join(predictions[0])
  cer_text_ref = "".join(references[0][0])
  cer_text_ori = cer_text_ori.replace("_", "")
  cer_text_ori = cer_text_ori.replace("▁", "")
  cer_text_pred = cer_text_pred.replace("_", "")
  cer_text_pred = cer_text_pred.replace("▁", "")
  cer_text_ref = cer_text_ref.replace("_", "")
  cer_text_ref = cer_text_ref.replace("▁", "")
  # print(f"ORI: {cer_text_ori}")
  # print(f"PRED: {cer_text_pred}")
  # print(f"REF: {cer_text_ref}")
  cer_original = cer(cer_text_ori, cer_text_ref)
  total_cer_ori += cer_original
  cer_pred = cer(cer_text_pred, cer_text_ref)
  total_cer_pred += cer_pred

  # print(f"GLUE PREDICTED: {gleu_score}")
  # print(f"GLUE ORIGINAL: {gleu_score_original}")
  # print(f"CER PREDICTED: {cer_pred}")
  # print(f"CER ORIGINAL: {cer_original}")
  # print(f"ACC: {acc}")
  # print(f"f1: {f1}")
  # print(f"ORIGINAL : {bleu_original}")
  # print(f"PREDICTED: {bleu_prediction}")

  # print("\n-----------------------")
print(f"AVG ACC: {float(totalacc/NUM_SAMPLE)}")
print(f"AVG f1: {float(total_f1/NUM_SAMPLE)}")
print(f"# HIGHER BLEU PREDICTION: {bleu_higher}")
print(f"# LOWER BLEU PREDICTION: {bleu_lower}")
print(f"# EQUAL BLEU PREDICTION: {bleu_equal}")
print(f"# GLEU PREDICTION: {float(total_gs_pred/NUM_SAMPLE)}")
print(f"# GLEU ORIGINAL: {float(total_gs_ori/NUM_SAMPLE)}")
print(f"# CER PREDICTION: {float(total_cer_pred/NUM_SAMPLE)}")
print(f"# CER ORIGINAL: {float(total_cer_ori/NUM_SAMPLE)}")

 40%|███▉      | 1985/5000 [1:22:27<2:51:15,  3.41s/it]

In [ ]:
class BertModel(torch.nn.Module):
    def __init__(self):
        super(BertModel, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', num_labels=2)
        self.bert.resize_token_embeddings(len(tokenizer))

    def forward(self, input_id, mask, label):
        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)
        return output

FILE = "drive/MyDrive/AIBuilders/mlm/tagging_nova_75.pth"
tagging_model = BertModel()
tagging_model.load_state_dict(torch.load(FILE, map_location=torch.device('cpu')))
tagging_model.eval()

In [ ]:
ids_to_labels = {0: 'f', 1: 'i'}

def evaluate_one_text(model, sentence, mask, labels):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()

    input_id = torch.Tensor([sentence]).type(torch.int64)
    label_ids = []
    for i in sentence:
      if i == 1 or i == 5 or i == 6:
        label_ids.append(-100)
      else:
        label_ids.append(2)
    label_ids = torch.Tensor([label_ids]).type(torch.int64)
    mask = torch.Tensor([mask]).type(torch.int64)

    logits = tagging_model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    return prediction_label

In [ ]:
bleu = load_metric("bleu")
totalacc = 0
bleu_higher = 0
bleu_lower = 0
bleu_equal = 0
total_f1 = 0

for i in tqdm(range(NUM_SAMPLE)):
  text_id = ds_tag.iloc[i]['text']['input_ids'].squeeze(0).tolist()
  text_token = tokenizer.convert_ids_to_tokens(text_id)
  original_token = text_token.copy()
  text_token_len = len(text_token)
  original = [original_token]
  labels_id = ds_mlm.iloc[i]['labels']
  mask = ds_mlm.iloc[i]['attention_mask']
  labels_token = tokenizer.convert_ids_to_tokens(labels_id)
  references = [[labels_token]]

  i_f = evaluate_one_text(tagging_model, text_id, mask, labels_id)
  print(i_f)
  i_f_len = len(i_f)

  print(f"NO. {i}, {''.join(text_token)}")

  for j in range(i_f_len):
    text_token[j+1] = text_token[j+1].replace("▁", " ")
    if(i_f[j] == 'i'):    
      if langid.classify(text_token[j+1])[0] == 'th':
        try:
          corrected = spell(text_token[j+1],"th_TH")
        except:
          print(f"err hunspell: {text_token[j+1]}")
        else:
          if not corrected == "No Suggestions":
            print(f"{text_token[j+1]} => {corrected[0]}")
            text_token[j+1] = corrected[0]
  predictions = [text_token]
  final = "".join(text_token)
  final = final.replace("▁", " ")

  numer = 0
  denom = 0
  TP = 0
  FP = 0
  TN = 0
  FN = 0
  print(len(text_token), len(labels_token))
  print(text_token)
  print(labels_token)
  labels_len = len(labels_token)
  for i in range(labels_len):
    if not text_token[i] == original_token[i]: #change
      denom += 1
      if text_token[i] == labels_token[i]:
        numer += 1
        TP += 1
      elif not text_token[i] == labels_token[i]:
        FP += 1
    elif text_token[i] == original_token[i]: #no change
      if text_token[i] == labels_token[i]:
        TN += 1
      elif not text_token[i] == labels_token[i]:
        FN += 1
  if denom == 0:
    acc = 0
  else:
    acc = float(numer)/float(denom)
  totalacc += acc

  print(f"TP:{TP}   TN:{TN}   FP:{FP}   FN:{FN}")
  precision = float(TP) / float(TP+FP) if TP+FP > 0 else 0
  recall = float(TP) / float(TP+FN) if TP+FN > 0 else 0
  f1 = float(2*precision*recall) / float(precision + recall) if precision+recall > 0 else 0
  total_f1 += f1

  bleu_original = bleu.compute(predictions=original, references=references)
  bleu_prediction = bleu.compute(predictions=predictions, references=references)

  if bleu_prediction['bleu'] > bleu_original['bleu']:
    bleu_higher += 1
  elif bleu_prediction['bleu'] < bleu_original['bleu']:
    bleu_lower += 1
  elif bleu_prediction['bleu'] == bleu_original['bleu']:
    bleu_equal += 1
  
  print(f"ACC: {acc}")
  print(f"f1: {f1}")
  print(f"ORIGINAL : {bleu_original}")
  print(f"PREDICTED: {bleu_prediction}")

  print("\n-----------------------")
print(f"AVG ACC: {float(totalacc/NUM_SAMPLE)}")
print(f"AVG f1: {float(total_f1/NUM_SAMPLE)}")
print(f"# HIGHER BLEU PREDICTION: {bleu_higher}")
print(f"# LOWER BLEU PREDICTION: {bleu_lower}")
print(f"# EQUAL BLEU PREDICTION: {bleu_equal}")